In [1]:
!pip install transformers>=4.32.0 optimum>=1.12.0 kaleido openai tiktoken python-multipart typing-extensions
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 14.8 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# from langchain.llms import HuggingFacePipeline
# llm = HuggingFacePipeline(pipeline=pipe)

model_name_or_path = "TheBloke/Llama-2-7b-Chat-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-64g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [7]:
!pip install gradio cohere type-extensions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 433.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 17.7 MB/s eta 0:00:00


In [8]:
import gradio as gr
def generate_response(prompt):
  # llm = HuggingFacePipeline(pipeline=pipe)

  prompt_template=f'''[INST] <<SYS>>
  You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
  Please ensure that your responses are socially unbiased and positive in nature.
  If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
  If you don't know the answer to a question, please don't share false information.
  <</SYS>>
  {prompt}[/INST]

  '''

  print("\n\n*** Generate:")

  pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
  )

  print(pipe(prompt_template)[0]['generated_text'])
  return pipe(prompt_template)[0]['generated_text']

def my_chatbot(input, history):
    history = history or []
    my_history = list(sum(history, ()))
    my_history.append(input)
    my_input = ' '.join(my_history)
    output = generate_response(my_input)
    history.append((input, output))
    return history, history

ImportError: ignored

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("""<h1><center>My Chatbot</center></h1>""")
    chatbot = gr.Chatbot()
    state = gr.State()
    txt = gr.Textbox(show_label=False, placeholder="Ask me a question and press enter.").style(container=False)
    txt.submit(my_chatbot, inputs=[txt, state], outputs=[chatbot, state])

demo.launch(share = True)